## Load Data and Initial Cleanup

In [18]:
import pandas as pd
df = pd.read_csv("logs_dataset.csv")

# Remove ordinal suffixes from the @timestamp string (e.g., "January 1st" becomes "January 1")
df['@timestamp'] = (
    df['@timestamp']
    .astype(str)
    .str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)
)

df.head()

,@timestamp,_id,ip_address
0,"July 8 2019, 14:43:03.000",XswJ0msBoTGddM7vxMDB,10.1.1.285
1,"July 8 2019, 14:43:01.000",dKQJ0msB7mP0GwVzvJjz,10.1.2.389
2,"July 8 2019, 14:42:59.000",CcwJ0msBoTGddM7vtb8y,10.1.1.415
3,"July 8 2019, 14:42:57.000",bKQJ0msB7mP0GwVzrZdT,10.1.1.79
4,"July 8 2019, 14:42:55.000",L6QJ0msB7mP0GwVzpZeI,10.1.1.60


## Convert to Datetime and Sort

In [19]:
df['@timestamp'] = pd.to_datetime(
    df['@timestamp'],
    format='%B %d %Y, %H:%M:%S.%f',
    errors='coerce'
)

# Sort by ip_address then timestamp to ensure sequential log analysis
df.sort_values(['ip_address', '@timestamp'], inplace=True)

df.head()

,@timestamp,_id,ip_address
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1


## Calculate Time Differences

In [20]:
# Shift timestamps within each IP group to get the previous event time
df['shift_time'] = df.groupby('ip_address')['@timestamp'].shift(1)

# Calculate difference in minutes
df['time_diff'] = (
    (df['@timestamp'] - df['shift_time'])
    .dt.total_seconds() // 60
)

df.head()

,@timestamp,_id,ip_address,shift_time,time_diff
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT,NaN
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09,82.0
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39,104.0
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49,0.0
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45,28.0


## Extract Temporal Features

In [21]:
# Create basic time features
df['date'] = df['@timestamp'].dt.date
df['weekday'] = df['@timestamp'].dt.dayofweek
df['hour'] = df['@timestamp'].dt.hour

# Identify if the log occurred on a weekend (Saturday=5, Sunday=6)
df['is_weekend'] = ((df['weekday'] == 5) | (df['weekday'] == 6)).astype(int)

# Create 4-hour buckets for coarser time analysis
df['hour_bucket'] = df['hour'] // 4

df.head()

,@timestamp,_id,ip_address,shift_time,time_diff,date,weekday,hour,is_weekend,hour_bucket
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT,NaN,2019-06-09,6,0,1,0
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09,82.0,2019-06-09,6,1,1,0
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39,104.0,2019-06-09,6,3,1,0
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49,0.0,2019-06-09,6,3,1,0
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45,28.0,2019-06-09,6,3,1,0


## Aggregate Logs by IP

In [22]:
# Count occurrences per IP address
ip_addr = 'ip_address'
ip_counts = df.groupby(ip_addr)['@timestamp'].count().reset_index()
ip_counts.columns = ['ip_address', 'log_count']

# Display the top 5 most active IPs
ip_counts.sort_values('log_count', ascending=False).head()

,ip_address,log_count
370,10.1.2.63,4372
255,10.1.2.249,4353
177,10.1.1.63,4339
311,10.1.2.386,4326
164,10.1.1.486,4317


## Rename Aggregate Counts

In [23]:
ip_counts = ip_counts.rename(columns={'@timestamp': 'total_count'})

ip_counts.head()

,ip_address,log_count
0,10.1.1.1,1446
1,10.1.1.100,2860
2,10.1.1.101,1465
3,10.1.1.106,1408
4,10.1.1.109,1459


## Calculate Daily Log Volume

In [24]:
# Group by IP and Date to find activity per day
daily_counts = (
    df.groupby([ip_addr, 'date'])['@timestamp']
    .count()
    .reset_index()
    .rename(columns={'@timestamp': 'daily_counts'})
)

daily_counts.head()

,ip_address,date,daily_counts
0,10.1.1.1,2019-06-09,36
1,10.1.1.1,2019-06-10,37
2,10.1.1.1,2019-06-11,70
3,10.1.1.1,2019-06-12,38
4,10.1.1.1,2019-06-13,32


## Find Median Daily Activity

In [25]:
daily_counts_avg = (
    daily_counts
    .groupby(ip_addr)['daily_counts']
    .median()
    .reset_index()
)

daily_counts_avg.head()

,ip_address,daily_counts
0,10.1.1.1,40.0
1,10.1.1.100,78.0
2,10.1.1.101,40.0
3,10.1.1.106,35.5
4,10.1.1.109,42.5


## Compare Weekend vs Weekday Activity

In [26]:
weekend_counts = (
    df.groupby([ip_addr, 'is_weekend'])['@timestamp']
    .count()
    .reset_index()
)

weekend_counts.head()

,ip_address,is_weekend,@timestamp
0,10.1.1.1,0,975
1,10.1.1.1,1,471
2,10.1.1.100,0,1960
3,10.1.1.100,1,900
4,10.1.1.101,0,1006
